In [12]:
import numpy as np
from PIL import Image
from scipy import signal, ndimage, misc
import  matplotlib.pyplot as plt
import sys
import itertools

'''
References:
    https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf
    http://homepages.inf.ed.ac.uk/rbf/AVINVERTED/STEREO/av5_siftf.pdf
    https://www.ipol.im/pub/art/2014/82/article_lr.pdf
'''

DIAG = list(set(itertools.permutations([-1,-1,1,1,0,0], 2)))

def is_extrema(x, y, down, mid, up):
    # 26 comparaisons to check if the point is a extrema (min or max)
    mymin = mymax = mid[x,y]
    rootx, rooty = x, y
    for xx, yy in DIAG:
        x = rootx + xx
        y = rooty + yy

        # out of bounds
        if x < 0 or y < 0 or x >= mid.shape[0] or y >= mid.shape[1]:
            return False

        mymin = min(down[x,y], mid[x,y], up[x,y], mymin)
        mymax = max(down[x,y], mid[x,y], up[x,y], mymax)

    # check if x is still the max or min
    return mymin != mid[rootx,rooty] and mymax == mid[rootx,rooty]

'''
The keypoint features are defined in SIFT as the extrema of the normalized
Laplacian scale-space 
'''
def locate_extremum(infos):
    for key in infos:
        infos[key]["kps"] = []
        pictures = infos[key]["dog"]
        for idx in range(1, len(pictures) -1):
            pic = pictures[idx]
            h, w = pic.shape
            for i in range(h):
                for j in range(w):
                    # If it is a local minimum or maximum
                    if is_extrema(i,j, pictures[idx-1], pic,
                                        pictures[idx+1]):

                       value = pic[i,j]
                       sup_pic, sub_pic= pictures[idx+1], pictures[idx-1]

                       # Computing dx_matrix -> 3d gradient
                       dx = (pic[i,j+1] - pic[i,j-1]) * .5 / 255
                       dy = (pic[i+1,j] - pic[i-1,j]) * .5 / 255
                       dt = (sup_pic[i,j]- sub_pic[i,j]) * .5 / 255
                       dx_matrix = np.matrix([[dx],[dy],[dt]])

                       # Computing Hessian matrix
                       dxx = (pic[i,j+1] + pic[i,j-1] - 2 * value) / 255
                       dyy = (pic[i+1,j] + pic[i-1,j] - 2 * value) / 255
                       dtt  = (sup_pic[i,j] + sub_pic[i,j] - 2 * value) / 255
                       dxy = (pic[i+1,j+1] - pic[i+1,j-1] - pic[i-1,j+1] + pic[i-1,j-1]) * 0.25  / 255
                       dxt = (sup_pic[i,j+1] - sup_pic[i,j-1] - sub_pic[i,j+1] + sub_pic[i,j-1])* 0.25 / 255
                       dyt = (sup_pic[i+1,j] - sup_pic[i-1,j] - sub_pic[i+1,j] + sub_pic[i-1,j]) * 0.25 / 255
                       
                       h1 = [dxx,dxy,dxt]
                       h2 = [dxy,dyy,dyt]
                       h3 = [dxt,dyt,dtt]
                       hessian_matrix = np.matrix([h1,h2,h3])

                       # Predict DoG value at subpixel extrema
                       try:
                           opt_X = -np.linalg.inv(hessian_matrix) @ dx_matrix
                       except:
                           continue
                           
                       # Low contrast extrema prunning
                       p = np.absolute(value + .5 * (dx_matrix.T @ opt_X))
                       detH2 = (dxx * dyy) - (dxy ** 2)
                       traceH2 = dxx + dyy
                       if p < .03 or detH2 <= 0 \
                           or (traceH2 ** 2) / detH2 > 12 \
                           or np.count_nonzero(opt_X < .5)  != 3:
                           continue

                       infos[key]["kps"].append((i,j))
             
# Show contours, approximation of laplacian
def diff_gaussian(infos, show=False):
    for key in infos:
        infos[key]["dog"] = []
        pictures = infos[key]["gaussian"]
        for idx in range(1, len(pictures)):
           pic1 = pictures[idx].astype('float64')
           pic2 = pictures[idx-1].astype('float64')
           pic_gauss = (pic1 - pic2)
           infos[key]["dog"].append(pic_gauss)

    if show:
        j = 1
        for key in infos:
            for picture in infos[key]["dog"]:
                plt.subplot(len(infos), len(infos[key]["dog"]), j)
                plt.imshow(picture, cmap="gray")
                j += 1
        plt.show()


# Creating linear scale space
def scale_space(img, infos, show=False):
    '''
        s: number of pictures
        k: constant factor for each adjacents scales
    '''
    
    s = 5
    nb_octave = 4
    k = np.power(2, 1/(s-1))
    std = np.sqrt(.5)
    # The first picture is a upsampling
    image = misc.imresize(img, 200, 'bilinear')
    
    for octave in range(1, nb_octave + 1):
        infos[octave] = {"gaussian": [], "std": []}
        infos[octave]['original'] = image
        # Create scale space 
        for i in range(s):
            new_std = std * np.power(k,i)
            blurred = ndimage.filters.gaussian_filter(image, new_std)
            infos[octave]["std"].append(new_std)
            infos[octave]["gaussian"].append(blurred)
         
        # Updating std
        std = std * np.power(k, 2) 
        
        # Resizing the picture
        image = misc.imresize(image, 50, 'bilinear') 
        
    if show:
        for key in infos:
            j = 1
            for blurred_image in infos[key]["gaussian"]:
                plt.subplot(1, s, j)
                plt.imshow(blurred_image, cmap="gray")
                j += 1
            plt.show()


def run(img):
    infos = {}
    scale_space(img, infos)
    diff_gaussian(infos)
    locate_extremum(infos)
    return infos


In [14]:
import os
import pickle
import itertools
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

from scipy.stats import multivariate_normal

'''
References:
    http://aishack.in/tutorials/sift-scale-invariant-feature-transform-keypoint-orientation/
    http://www.vlfeat.org/api/sift.html
'''

'''
    infos: dict_keys([1, 2, 3, 4])
    infos[1]: dict_keys(['original', 'gaussian', 'std' , 'dog', 'kps'])
    
    infos[1]["gaussian"] = [img0, img1...]
    infos[1]["kps"] = [tuple, tuple1...]
'''


'''
    The size of the "orientation collection region" around the keypoint depends
    on
    it's scale. The bigger the scale, the bigger the collection region.
'''

### Compute gradient magnitude
def gradient_m(i,j, picture):
    ft = (picture[i,j+1] - picture[i,j-1]) ** 2
    st = (picture[i+1,j] - picture[i-1,j]) ** 2

    return np.sqrt(ft + st)

### Compute gradient orientation
def gradient_theta(i,j,picture):
    #To avoid error division by zero
    eps = 1e-5
    
    ft = picture[i+1,j] - picture[i-1,j]
    st = (picture[i,j+1] - picture[i,j-1]) + eps

    ret = 180 + np.arctan2(ft, st)  * 180 / np.pi
    return ret


def dominant_hist(hist):
    sorted_hist = np.argsort(hist, axis=0)[::-1]
    max_hist = hist[sorted_hist[0]]
    i = 1
    while .8 * max_hist < hist[sorted_hist[i]]:
        print("Dominant i arg {}".format(i))
        i += 1


def create_histogram(i,j,picture,std):
    truncate = 4.0
    kernel_size = 2 * int(std * truncate + .5) + 1
    window  = list(range(-kernel_size, kernel_size + 1))

    diag  = set(itertools.permutations(window, 2))
    rooti, rootj = i,j
    theta_list =  []

    gaussian = multivariate_normal(mean=[i,j], cov=1.5*std)
    orient_hist = np.zeros([36,1])
    
    for ii, jj in diag:
        x = rooti + ii
        y = rootj + jj
        if x - 1 < 0 or y - 1 < 0 or x + 1 > picture.shape[0] - 1 \
            or y + 1 > picture.shape[1] -1:
            continue
        
        # TODO: Warning the magnitude are really small
        magnitude = gradient_m(x,y,picture)
        weight =  magnitude * gaussian.pdf([x,y])

        orientation = gradient_theta(x,y,picture)
        bins_orientation = np.clip(orientation // 10, 0,35)

        orient_hist[int(bins_orientation)] += weight

    return orient_hist


def assign_orientation(infos):
    index = 0 
    for octave in infos.keys():
        kps = infos[octave]['kps']
        std = infos[octave]["std"][index]
        picture = infos[octave]['gaussian'][index].astype('float64')

        for i, j in kps: 
            hist = create_histogram(i,j,picture,std)
            dominant_hist(hist)
        print("Next octave")
    return infos

### Showing keypoints for the first octave's picture
def show_keypoints(infos):
    #pic = np.zeros(infos[1]['gaussian'][0].shape)
    pic = infos[1]['gaussian'][0]
    for x,y in infos[1]['kps']:
        pic[x,y] = 255
    
    plt.imshow(pic, cmap="gray")
    plt.show()

def run(load=None, img=None):
    infos = None
    if load  is not None:
        infos = pickle.load(open(load, "rb"))
        assign_orientation(infos)
        #show_keypoints(infos)
    if img is not None:
        infos = laplacian.run(img) 
    return infos 

def reload():
    def step(path, name):
        img = np.array(Image.open(path).convert('L'))
        infos = run(img=img)
        pickle.dump(infos, open("pickle/infos_{}.pickle".format(name), "wb"))
        
    path_paris = 'paris.jpg'
    path_cat = 'cat.jpg'
    
    step(path_paris, "paris")
    step(path_cat, "cat")

if __name__ == "__main__":
    path_paris = 'paris.jpg'
    path_cat = 'cat.jpg'
    img = np.array(Image.open(path_cat).convert('L'))
    run()

    #reload()